In [ ]:
import pandas as pd
import numpy as np
import json
import sys
import random

import ast

import pymysql

from datetime import date

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import matplotlib.patheffects as path_effects
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import to_rgba
from matplotlib import cm
from matplotlib import colorbar
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.patches import RegularPolygon
from matplotlib.patches import ArrowStyle
from matplotlib.patches import Circle

from matplotlib.colors import Normalize
import matplotlib.patheffects as pe

import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from mplsoccer import VerticalPitch, Pitch, Radar, FontManager, grid, PyPizza

import scipy.stats as stats

from highlight_text import  ax_text, fig_text

from soccerplots.utils import add_image

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing

import abs_import

import statistics
import math
import ipywidgets as widgets

from IPython.display import display, Math, Latex

import warnings

plt.rcParams["figure.dpi"] = 300

from matplotlib import font_manager

from app import *

sys.path.append('Functions/visualization')

from Functions.utils import read_json
from Functions.visualization.passing_network import draw_pitch, draw_pass_map

clubColors = {'Brazil' : ['#fadb04', '#1c3474'],
              'Portugal' : ['#e1231b', '#004595'],
              'Argentina' : ['#52a9dc', '#dbe4ea'],
              'Saudi Arabia' : ['#145735', '#dbe4ea'],
              'Ghana' : ['#145735', '#dbe4ea'],
              'Serbia' : ['#FF0000', '#E8E8E8'],
              'Spain' : ['#FF0000', '#E8E8E8'],
              'Germany' : ['#aa9e56', '#FF0000'],
              'France' : ['#202960', '#d10827'],
              'Poland' : ['#d10827', '#E8E8E8'],
              'Morocco' : ['#db221b', '#044c34'],
              'Croatia' : ['#e71c23', '#3f85c5'],
              'Netherlands' : ['#f46c24', '#dcd9d7'],
              'Senegal' : ['#34964a', '#eedf36'],
              'Denmark' : ['#cb1617', '#E8E8E8'],
              'Iran' : ['#269b44', '#dd1212'],
              'Belgium' : ['#ff0000', '#e30613'],
              'USA' : ['#ff0000', '#202960'],
              'Switzerland' : ['#ff0000', '#e8e8e8'],
              'Australia' : ['#202960', '#e30613'],
              'Wales' : ['#ff0000', '#e8e8e8'],
              'Mexico' : ['#00a94f', '#ff0000'],
              'Uruguay' : ['#52a9dc', '#e8e8e8'],
              'Canada' : ['#ff0000', '#ff0000'],
              'Costa Rica' : ['#ff0000', '#202960']}

df1 = pd.read_csv('homeGameformations.csv')
df1.rename({'playerIds' : 'playerId'}, axis=1, inplace=True)
df2 = pd.read_csv('optaMundial.csv')
df2["matchTimestamp"] = 60 * df2["minute"] + df2["second"]
df2["matchTimestamp"] = pd.to_datetime(df2["matchTimestamp"], unit='s')
df2.drop_duplicates(subset=['name', 'matchTimestamp', 'team', 'typedisplayName', 'x', 'y'], keep='first', inplace=True)
df2.sort_values(by=['Match_ID', 'matchTimestamp'], inplace=True, ascending=[True, True])
df2.reset_index(drop=True, inplace=True)

In [ ]:
# CREATE FUNCTION TO SPLIT VALUES FROM A LIST SEPARETED WITH ,
def pad_list(lst, max_len):
    return lst + [lst[-1]] * (max_len - len(lst))

# SELECT THE COLUMNS WE WANT TO TRANSFORM
df3 = df1[['formationId', 'jerseyNumbers', 'formationName', 'playerId', 'formationSlots', 'startMinuteExpanded', 'endMinuteExpanded']]

# CONVERT COLUMNS TO STRING
df3[['startMinuteExpanded', 'endMinuteExpanded', 'formationId']] = df3[['startMinuteExpanded', 'endMinuteExpanded', 'formationId']].astype(str)

# APPLY FUNCTION TO SPLIT VALUES FROM A LIST SEPARETED WITH , AND EXPLODE THEM AND CONVERT TO LIST
new_df = df3[['formationSlots', 'startMinuteExpanded', 'endMinuteExpanded', 'formationId', 'jerseyNumbers', 'playerId']].apply(lambda row: row.apply(lambda item: pad_list(item.split(','), max(row.str.contains(',') + 1))), axis=1).explode(df3[['formationSlots', 'jerseyNumbers', 'playerId']].columns.tolist())

# REMOVE ONE CARATHER FROM INSIDE A COLUMN VALUE
new_df['formationSlots'] = new_df['formationSlots'].str.replace("[[]","")
new_df['formationSlots'] = new_df['formationSlots'].str.replace("[]]","")

new_df['jerseyNumbers'] = new_df['jerseyNumbers'].str.replace("[[]","")
new_df['jerseyNumbers'] = new_df['jerseyNumbers'].str.replace("[]]","")

new_df['playerId'] = new_df['playerId'].str.replace("[[]","")
new_df['playerId'] = new_df['playerId'].str.replace("[]]","")

# CONVERT COLUMNS TO INTEGER
new_df['formationSlots'] = new_df['formationSlots'].astype(int)
new_df['jerseyNumbers'] = new_df['jerseyNumbers'].astype(int)
new_df['playerId'] = new_df['playerId'].astype(int)

# GET FIRST ELEMENT FROM A COLUMN LIST
new_df['minute'] = new_df.startMinuteExpanded.apply(lambda x: x[0])
new_df['endMinuteExpanded'] = new_df.endMinuteExpanded.apply(lambda x: x[0])
new_df['formationId'] = new_df.formationId.apply(lambda x: x[0])

new_df[['formationId', 'formationSlots', 'minute', 'endMinuteExpanded', 'jerseyNumbers', 'playerId']] = new_df[['formationId', 'formationSlots', 'minute', 'endMinuteExpanded', 'jerseyNumbers', 'playerId']].astype(int)

# REMOVE VALUES EQUAL TO 0
new_df = new_df.loc[new_df.formationSlots != 0].reset_index(drop=True)

# CREATE COLUMN FORMATION BASED ON IF CONDITION
new_df['formationName'] = 'tactic'
for idx, row in enumerate(new_df.formationId):
    if row == 2:
        new_df['formationName'][idx] = '442'
    elif row == 3:
        new_df['formationName'][idx] = '41212 (Diamond)'
    elif row == 4:
        new_df['formationName'][idx] = '433'
    elif row == 5:
        new_df['formationName'][idx] = '451'
    elif row == 6:
        new_df['formationName'][idx] = '4411'
    elif row == 7:
        new_df['formationName'][idx] = '4141'
    elif row == 8:
        new_df['formationName'][idx] = '4231'
    elif row == 9:
        new_df['formationName'][idx] = '4321'
    elif row == 10:
        new_df['formationName'][idx] = '532'
    elif row == 11:
        new_df['formationName'][idx] = '541'
    elif row == 12:
        new_df['formationName'][idx] = '352'
    elif row == 13:
        new_df['formationName'][idx] = '343'
    elif row == 14:
        new_df['formationName'][idx] = '31312'
    elif row == 15:
        new_df['formationName'][idx] = '4222'
    elif row == 16:
        new_df['formationName'][idx] = '3511'
    elif row == 17:
        new_df['formationName'][idx] = '3421'
    elif row == 18:
        new_df['formationName'][idx] = '3412'        


# CREATE COLUMN POSITION BASED ON TWO IF CONDITION
new_df['position'] = 'POS'

# formationId 2: 4-4-2
for idx, row in enumerate(new_df.formationSlots):
    if (row == 1) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'RCM'
    elif (row == 5) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'RM'
    elif (row == 8) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'LCM'
    elif (row == 9) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'LCF'
    elif (row == 10) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'RCF'
    elif (row == 11) & (new_df.formationName[idx] == '442'):
        new_df['position'][idx] = 'LM'

    # formationId 3: 4-1-2-1-2 (Diamond)
    elif (row == 1) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'CDM'
    elif (row == 5) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'RCM'
    elif (row == 8) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'CAM'
    elif (row == 9) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'LCF'
    elif (row == 10) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'RCF'
    elif (row == 11) & (new_df.formationName[idx] == '41212 (Diamond)'):
        new_df['position'][idx] = 'LCM'

    # formationId 4: 4-3-3
    elif (row == 1) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'DMC'
    elif (row == 5) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RCM'
    elif (row == 8) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LCM'
    elif (row == 9) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'RW'
    elif (row == 11) & (new_df.formationName[idx] == '433'):
        new_df['position'][idx] = 'LW'

    # formationId 5: 4-5-1
    elif (row == 1) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'RDM'
    elif (row == 5) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'RM'
    elif (row == 8) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'LDM'
    elif (row == 9) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'CAM'
    elif (row == 11) & (new_df.formationName[idx] == '451'):
        new_df['position'][idx] = 'LM'

    # formationId 6: 4-4-1-1
    elif (row == 1) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'RCM'
    elif (row == 5) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'RM'
    elif (row == 8) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'LCM'
    elif (row == 9) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'CAM'
    elif (row == 11) & (new_df.formationName[idx] == '4411'):
        new_df['position'][idx] = 'LM'

    # formationId 7: 4-1-4-1
    elif (row == 1) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'CDM'
    elif (row == 5) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'RM'
    elif (row == 8) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'RCM'
    elif (row == 9) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'LCM'
    elif (row == 11) & (new_df.formationName[idx] == '4141'):
        new_df['position'][idx] = 'LM'

    # formationId 8: 4-2-3-1
    elif (row == 1) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] =  'LDM'
    elif (row == 5) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'RW'
    elif (row == 8) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'RDM'
    elif (row == 9) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'CAM'
    elif (row == 11) & (new_df.formationName[idx] == '4231'):
        new_df['position'][idx] = 'LW'

    # formationId 9: 4-3-2-1
    elif (row == 1) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'CDM'
    elif (row == 5) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'LCM'
    elif (row == 8) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'RCM'
    elif (row == 9) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'RAM'
    elif (row == 11) & (new_df.formationName[idx] == '4321'):
        new_df['position'][idx] = 'LAM'

    # formationId 10: 5-3-2
    
    elif (row == 1) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'LCB'
    elif (row == 5) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'RCM'
    elif (row == 8) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'CDM'
    elif (row == 9) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'LCF'
    elif (row == 10) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'RCF'
    elif (row == 11) & (new_df.formationName[idx] == '532'):
        new_df['position'][idx] = 'LCM'

    # formationId 11: 5-4-1
    elif (row == 1) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'LWB'
    elif (row == 4) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'LB'
    elif (row == 5) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'RM'
    elif (row == 8) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'RCM'
    elif (row == 9) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'LCM'
    elif (row == 11) & (new_df.formationName[idx] == '541'):
        new_df['position'][idx] = 'RM'

    # formationId 12: 3-5-2
    elif (row == 1) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'RWB'
    elif (row == 3) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'LWB'
    elif (row == 4) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'LCB'
    elif (row == 5) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'RDM'
    elif (row == 8) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'LDM'
    elif (row == 9) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'LCF'
    elif (row == 10) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'RCF'
    elif (row == 11) & (new_df.formationName[idx] == '352'):
        new_df['position'][idx] = 'CAM'
        
    # formationId 13: 3-4-3
    elif (row == 1) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'RWB'
    elif (row == 3) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'LWB'
    elif (row == 4) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'LCB'
    elif (row == 5) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'RCM'
    elif (row == 8) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'LCM'
    elif (row == 9) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'RW'
    elif (row == 11) & (new_df.formationName[idx] == '343'):
        new_df['position'][idx] = 'LW'

    # formationId 14: 3-1-3-1-2
    elif (row == 1) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'RCM'
    elif (row == 3) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'LCM'
    elif (row == 4) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'CDM'
    elif (row == 5) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'LCB'
    elif (row == 8) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'CM'
    elif (row == 9) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'RCF'
    elif (row == 10) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'CAM'
    elif (row == 11) & (new_df.formationName[idx] == '31312'):
        new_df['position'][idx] = 'LCF'
        
    # formationId 15: 4-2-2-2
    elif (row == 1) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'RB'
    elif (row == 3) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'LB'
    elif (row == 4) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'RDM'
    elif (row == 5) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'RCB'
    elif (row == 6) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'LCB'
    elif (row == 7) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'LDM'
    elif (row == 8) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'RCM'
    elif (row == 9) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'LCF'
    elif (row == 10) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'RCF'
    elif (row == 11) & (new_df.formationName[idx] == '4222'):
        new_df['position'][idx] = 'LCM'
        
    # formationId 16: 3-5-1-1

    elif (row == 1) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'RWB'
    elif (row == 3) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'LWB'
    elif (row == 4) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'LCB'
    elif (row == 5) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'RDM'
    elif (row == 8) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'LDM'
    elif (row == 9) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'CF'
    elif (row == 10) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'CAM'
    elif (row == 11) & (new_df.formationName[idx] == '3511'):
        new_df['position'][idx] = 'CM'
        
    # formationId 17: 3-4-2-1

    elif (row == 1) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'RWB'
    elif (row == 3) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'LWB'
    elif (row == 4) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'LCB'
    elif (row == 5) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'RCM'
    elif (row == 8) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'LCM'
    elif (row == 9) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'LAM'
    elif (row == 10) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'RAM'
    elif (row == 11) & (new_df.formationName[idx] == '3421'):
        new_df['position'][idx] = 'CF'

    # formationId 18: 3-4-1-2
    elif (row == 1) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'GK'
    elif (row == 2) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'RWB'
    elif (row == 3) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'LWB'
    elif (row == 4) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'LCB'
    elif (row == 5) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'CCB'
    elif (row == 6) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'RCB'
    elif (row == 7) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'RCM'
    elif (row == 8) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'LCM'
    elif (row == 9) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'CAM'
    elif (row == 10) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'RCF'
    elif (row == 11) & (new_df.formationName[idx] == '3412'):
        new_df['position'][idx] = 'LCF'
        
new_df = new_df[['formationId', 'formationSlots', 'minute', 'endMinuteExpanded', 'jerseyNumbers', 'playerId', 'formationName', 'position']]

new_df.head(11)

In [36]:
df3 = df2.loc[(df2.team == 'Portugal') & (df2.Match_ID == 13)].reset_index(drop=True)

In [38]:
df = pd.merge(df3, new_df, on=['minute', "playerId"])
df.rename({'minute_x' : 'minute', 'position' : 'Position'}, axis=1, inplace=True)

In [39]:
df.drop_duplicates(subset=['name', 'matchTimestamp', 'team', 'typedisplayName', 'x', 'y'], keep='first', inplace=True)
df.sort_values(by=['Match_ID', 'matchTimestamp'], inplace=True, ascending=[True, True])
df.reset_index(drop=True, inplace=True)
df = df[['matchTimestamp', 'name', 'Position', 'team', 'typedisplayName', 'outcomeTypedisplayName', 'x', 'y', 'endX', 'endY', 'home_Team', 'away_Team', 'Match_ID', 'formationName']]
df

,matchTimestamp,name,Position,team,typedisplayName,outcomeTypedisplayName,x,y,endX,endY,home_Team,away_Team,Match_ID,formationName
0,1970-01-01 00:00:07,Diogo Dalot,RB,Portugal,Aerial,Successful,35.6,28.2,0.0,0.0,Portugal,Switzerland,13,433
1,1970-01-01 00:00:07,Diogo Dalot,RB,Portugal,Pass,Unsuccessful,30.9,24.8,41.7,25.8,Portugal,Switzerland,13,433
2,1970-01-01 00:00:09,Bruno Fernandes,RW,Portugal,Aerial,Unsuccessful,45.3,25.0,0.0,0.0,Portugal,Switzerland,13,433
3,1970-01-01 00:00:10,William Carvalho,DMC,Portugal,Pass,Successful,34.3,28.9,29.8,7.1,Portugal,Switzerland,13,433
4,1970-01-01 00:00:12,Diogo Dalot,RB,Portugal,BallRecovery,Successful,29.8,7.1,0.0,0.0,Portugal,Switzerland,13,433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,1970-01-01 01:30:46,Pepe,RCB,Portugal,Pass,Successful,22.2,11.8,28.9,20.3,Portugal,Switzerland,13,433
58,1970-01-01 01:30:51,Vitinha,RCM,Portugal,Pass,Successful,41.6,25.4,48.0,9.4,Portugal,Switzerland,13,433
59,1970-01-01 01:30:53,Cristiano Ronaldo,CF,Portugal,Pass,Successful,46.8,11.5,37.9,14.0,Portugal,Switzerland,13,433
60,1970-01-01 01:30:56,Diogo Dalot,RB,Portugal,Pass,Successful,39.3,18.2,32.8,35.0,Portugal,Switzerland,13,433


In [40]:
df.loc[df.name == 'Bernardo Silva']['Position'].unique()

array(['LCM'], dtype=object)